In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
%load_ext autoreload
%autoreload 2
import numpy as np 
from copy import deepcopy
import lbforaging
import torch
from torch.autograd import Variable
# import robotic_warehouse
import gym
import time

# import seql code
from seac.seql.lbf_train_rs import LBFTrainRS
from seac.seql.iql import IQL

Using cuda device
Using cuda device
Using GPU


## Read logs

In [11]:
nagents=2
logs_dir = 'logs/'
path = "models/default/"
RUN_STEPS = 100

In [3]:
USE_CUDA = torch.cuda.is_available()
if USE_CUDA :
    print('Using GPU')

Using GPU


In [4]:
args=[ '--ltl', '(f & XF e)', 
      '--num_episodes', str(RUN_STEPS), 
      '--discount_oa', '0.8', 
      '--field', '5', 
      '--render', 'True',
     '--env_coef', '0.2']
trainer = LBFTrainRS(args)
print(trainer.arglist)

Namespace(agents=2, batch_size=128, buffer_capacity=1000000, discount_oa=0.8, env=None, env_coef=0.2, epsilon=1.0, epsilon_anneal_slow=False, epsilon_decay=10, eval_episodes=5, eval_frequency=50, field=5, gamma=0.99, goal_epsilon=0.01, hidden_dim=128, lr=0.0001, ltl='(f & XF e)', max_episode_len=25, n_training_threads=1, num_episodes=10, oa_coef=1.0, render=True, run='default', save_interval=100, seed=None, shared_experience=False, shared_lambda=1.0, steps_per_update=1, targets='simple', tau=0.05, training_returns_freq=100)


In [5]:
if trainer.arglist.max_episode_len == 25:
    steps = trainer.arglist.num_episodes * 20 #self.arglist.max_episode_len
else:
    steps = trainer.arglist.num_episodes * trainer.arglist.max_episode_len
# steps-th root of goal epsilon
if trainer.arglist.epsilon_anneal_slow:
    decay_factor = trainer.arglist.epsilon_decay ** (1 / float(steps))
    trainer.arglist.decay_factor = decay_factor
    print(
        f"Epsilon is decaying with (({trainer.arglist.epsilon_decay} - {decay_factor}**t) / {trainer.arglist.epsilon_decay}) to {trainer.arglist.goal_epsilon} over {steps} steps."
    )
else:
    decay_epsilon = trainer.arglist.goal_epsilon ** (1 / float(steps))
    trainer.arglist.decay_factor = decay_epsilon
    print( "Epsilon is decaying with factor %.7f to %.3f over %d steps."
        % (decay_epsilon, trainer.arglist.goal_epsilon, steps))

Epsilon is decaying with factor 0.9772372 to 0.010 over 200 steps.


In [7]:
env, env_name, task_name, n_agents, observation_spaces, action_spaces, observation_sizes, action_sizes = (
            trainer.create_environment()
        )

/root/.venvs/csrl/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Observation spaces:  Tuple(Box([-1. -1.  0. -1. -1.  0. -1. -1.  0.  0.], [4. 4. 4. 4. 4. 2. 4. 4. 2. 4.], (10,), float32), Box([-1. -1.  0. -1. -1.  0. -1. -1.  0.  0.], [4. 4. 4. 4. 4. 2. 4. 4. 2. 4.], (10,), float32))
Action spaces:  Tuple(Discrete(6), Discrete(6))


/root/.venvs/csrl/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [8]:
trainer.alg = IQL(n_agents, observation_sizes, action_sizes, trainer.arglist)
trainer.alg.load_model_networks(path, extension='_241000')

In [13]:
obs = trainer.reset_environment()

for i in range(RUN_STEPS):
    torch_obs = [Variable(torch.Tensor(obs[i]), requires_grad=False) for i in range(n_agents)]
    actions, onehot_actions= trainer.select_actions(torch_obs, explore = False)
    trainer.environment_render(step=i, inline=False)
    reward, done, nexty_obs, info = trainer.environment_step(actions)
    if all(done):
        obs = trainer.reset_environment()
        print("--- Episode Finished ---")
        print(f"Episode rewards: {sum(info['episode_reward'])}")
        print(info)
        print(" --- ")
        time.sleep(2.4)


print('all done')

--- Episode Finished ---
Episode rewards: 0.0
{'episode_reward': array([0., 0.]), 'agent0/episode_reward': 0.0, 'agent1/episode_reward': 0.0, 'episode_length': 25, 'episode_time': 2.6735904090019176}
 --- 
--- Episode Finished ---
Episode rewards: 1.0
{'episode_reward': array([0., 1.]), 'agent0/episode_reward': 0.0, 'agent1/episode_reward': 1.0, 'episode_length': 1, 'episode_time': 2.5093766919999325}
 --- 
--- Episode Finished ---
Episode rewards: 0.0
{'episode_reward': array([0., 0.]), 'agent0/episode_reward': 0.0, 'agent1/episode_reward': 0.0, 'episode_length': 25, 'episode_time': 5.057859708998876}
 --- 
--- Episode Finished ---
Episode rewards: 1.0
{'episode_reward': array([0., 1.]), 'agent0/episode_reward': 0.0, 'agent1/episode_reward': 1.0, 'episode_length': 11, 'episode_time': 3.574652256000263}
 --- 
--- Episode Finished ---
Episode rewards: 0.0
{'episode_reward': array([0., 0.]), 'agent0/episode_reward': 0.0, 'agent1/episode_reward': 0.0, 'episode_length': 25, 'episode_time':

In [14]:
trainer.env.close()